# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [1]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Step 1: Load Chess Data

Load cleaned data from csv file


In [2]:
df = pd.read_csv("C:/Users/anany/OneDrive/Documents/Chess-engine/bot/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2: Pre-processing

2.1 Turning features to numerical values (moves.. etc) 

2.2 Encoding the features to be trained

In [3]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [4]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[391, 1078, 690, 1585, 223, 1424, 1850, 667, 945, 1099]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [5]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [6]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [7]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 391, 1), (0, 1078, 0), (1, 690, 1), (0, 1585, 0), (1, 223, 1), (0, 1424, 0), (1, 1850, 1), (0, 667, 0), (1, 945, 1), (0, 1099, 0), (1, 1205, 1), (0, 521, 0), (1, 38, 1), (0, 331, 0), (1, 587, 1), (0, 665, 0), (1, 121, 1), (0, 1094, 0), (1, 1551, 1), (0, 1044, 0), (1, 1874, 1), (0, 1562, 0), (1, 769, 1), (0, 1523, 0), (1, 632, 1), (0, 1835, 0), (1, 267, 1), (0, 1794, 0), (1, 525, 1), (0, 1225, 0), (1, 1874, 1), (0, 1313, 0), (1, 1624, 1), (0, 1427, 0), (1, 541, 1), (0, 1512, 0), (1, 753, 1), (0, 963, 0), (1, 760, 1), (0, 984, 0), (1, 517, 1), (0, 729, 0), (1, 1861, 1), (0, 534, 0), (1, 541, 1), (0, 1626, 0), (1, 1721, 1), (0, 780, 0), (1, 322, 1), (0, 691, 0), (1, 161, 1), (0, 1789, 0), (1, 290, 1), (0, 1094, 0), (1, 1903, 1), (0, 1403, 0), (1, 541, 1), (0, 826, 0), (1, 760, 1), (0, 677, 0), (1, 1331, 1), (0, 1038, 0), (1, 852, 1), (0, 1468, 0), (1, 1157, 1), (0, 1287, 0), (1, 1106, 1), (0, 1182, 0), (1, 1331, 1), (0, 1287, 0), (1, 1106, 1), (0, 1182, 0), (1, 1757, 1), (0, 1442, 0)

In [8]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 391, 1), (0, 1078, 0), (1, 690, 1), (0, 1..."
1,123118510404,"[(1, 690, 0), (0, 462, 1), (1, 1425, 0), (0, 9..."
2,123118790014,"[(1, 391, 0), (0, 945, 1), (1, 1850, 0), (0, 1..."
3,123158939328,"[(1, 391, 0), (0, 1425, 1), (1, 945, 0), (0, 5..."
4,123160166430,"[(1, 391, 1), (0, 945, 0), (1, 585, 1), (0, 11..."


### Step 2.2:  Embedded layer Encoding 


In [11]:
# extract teoriat's moves per game
teoriat_sequences = []
for game_moves in df["game_data"]:
    teoriat_moves = [m[1] for m in game_moves if m[2] == 1]
    if len(teoriat_moves) > 5:
        teoriat_sequences.append(teoriat_moves)

# prep sequences for training
all_move_ids = sorted(list(set([m for seq in teoriat_sequences for m in seq])))
move_to_idx = {move: i for i, move in enumerate(all_move_ids)}
idx_to_move = {i: move for move, i in move_to_idx.items()}
vocab_size = len(move_to_idx)

# encode teoriat_moves as indices
encoded_sequences = [[move_to_idx[m] for m in seq] for seq in teoriat_sequences]

# create dataset for RNN
class ChessDataset(Dataset):
    def __init__(self, encoded_sequences, seq_len=10):
        self.seq_len = seq_len
        self.samples = []

        for seq in encoded_sequences:
            for i in range(len(seq) - seq_len):
                x = seq[i:i+seq_len]
                y = seq[i+seq_len]
                self.samples.append((x, y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

dataset = ChessDataset(encoded_sequences, seq_len=10)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f"Total training samples: {len(dataset)}")
print(f"Vocabulary size (unique moves): {vocab_size}")

Total training samples: 20966
Vocabulary size (unique moves): 1638


### Step 2.3: Model Definition

In [20]:
#model 
class ChessRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# Instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChessRNN(vocab_size).to(device)
print(model)

ChessRNN(
  (embedding): Embedding(1638, 128)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=1638, bias=True)
)


### Step 2.4: Training Loop

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = total = 0

    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total += y_batch.size(0)
        preds = output.argmax(dim=1)
        correct += (preds == y_batch).sum().item()

    avg_loss = total_loss / len(train_loader)
    train_acc = correct / total

    print(f"Epoch [{epoch+1}] | Loss: {avg_loss:.4f} | Accuracy: {train_acc:.4f}")

#Final summary
print("\n Final Training Results")
print(f"Final Training Loss: {avg_loss:.4f}")
print(f"Final Training Accuracy: {train_acc:.4f}")


Epoch [1] | Loss: 6.7419 | Accuracy: 0.0085
Epoch [2] | Loss: 6.2669 | Accuracy: 0.0183
Epoch [3] | Loss: 5.9711 | Accuracy: 0.0275
Epoch [4] | Loss: 5.6232 | Accuracy: 0.0426
Epoch [5] | Loss: 5.2113 | Accuracy: 0.0651
Epoch [6] | Loss: 4.7570 | Accuracy: 0.1020
Epoch [7] | Loss: 4.2926 | Accuracy: 0.1460
Epoch [8] | Loss: 3.8548 | Accuracy: 0.2000
Epoch [9] | Loss: 3.4382 | Accuracy: 0.2632
Epoch [10] | Loss: 3.0536 | Accuracy: 0.3216

 Final Training Results
Final Training Loss: 3.0536
Final Training Accuracy: 0.3216
